In [1]:
import pandas as pd
import math
from dateutil.relativedelta import relativedelta
from datetime import datetime , timedelta
import re
import copy
from sqlalchemy import create_engine
import calendar

In [2]:
conn_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(user='postgres',
                                                                     pwd='123',
                                                                     host='localhost',
                                                                     port='5432',
                                                                     db='postgres'
                                                                     )
engine = create_engine(conn_string, paramstyle='format', isolation_level="READ UNCOMMITTED")

In [3]:
path = 'D:/GA/GA-VH'
state = 'GA'

In [4]:
slab_fields = pd.read_excel(r'C:\Users\CDMA\Downloads\slab_fields_prep_v2.xlsx',sheet_name = 'slab_fields_prep')

In [5]:
slab_new = pd.read_csv(r'C:\Users\CDMA\Downloads\vm_tax_slab_new_transformed.csv')
slab_new = slab_new[slab_new['state_cd']==state]
slab_new = slab_new[slab_new['vh_class'].notnull()]

In [6]:
slab_new['date_from'] = slab_new['date_from'].astype('datetime64[ns]')
slab_new['date_to'] = slab_new['date_to'].astype('datetime64[ns]')
slab_new.reset_index(drop=True, inplace=True)

In [7]:
slab_add_new = pd.read_csv(r'D:\Jharkhand python\vm_tax_slab_add_new.csv')
slab_add_new = slab_add_new[slab_add_new['state_cd']==state]

In [8]:
vt_tax_based_on = pd.read_csv(path+'/vt_tax_based_on .csv')
#vt_tax_based_on_permit_info = pd.read_csv(r'D:\AS\vt_tax_based_on_permit_info.csv')
#vt_tax_based_on = pd.read_sql_query('Select * from vt_tax_based_on where state_cd ='+"'{}'".format(state),con=engine2)
vt_tax_based_on = vt_tax_based_on[vt_tax_based_on['state_cd']==state]

In [9]:
vm_vh_class=pd.read_csv(r'D:\Jharkhand python\vm_vh_class.csv')

In [10]:
vt_owner = pd.read_csv(path+'/vt_owner.csv')
vt_owner = vt_owner[vt_owner['state_cd']==state]
#vt_owner = pd.read_sql_query('Select * from vt_owner where state_cd ='+"'{}'".format(state),con=engine2)

C:\Users\CDMA\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (52) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [11]:
tm_office = pd.read_csv('D:/RJ/tm_office.csv')
#tm_office.head()
tm_office = tm_office[tm_office['state_cd']==state]

In [12]:
#Turn into first letter capital in each word
def change_col_names(cols):
    pp=[]
    for j in cols :
        p= j.replace('_',' ').title()
        pp.append(p)
        #df.rename(columns={j: p},inplace=True)
    return pp

In [13]:
def calculate_formula(formula_df,params_info):
    #Calculate the Formula one by one
    ind=[]
    cff_formula = []
    cc_formula=[]
    ccf_result=[]
    cr_rate=[]
    #c_add_code=[]
    #reb_df = formula_df[formula_df['type_flag']=='R']
    for l in range(len(formula_df)):
        #print('l',l)
        condition_formula = formula_df['condition_formula'].to_list()[l]
        #print(condition_formula)
        if '\r' in condition_formula:
            condition_formula = condition_formula.replace('\r','')
        if '\n' in condition_formula:
            condition_formula = condition_formula.replace('\n','')

        cf_params = []
        for k in re.split(r'[()\*\s]\s*',condition_formula):
            if '<' in k and '>' in k:
                cf_params.append(k[k.find('<'):(k.find('>')+1)])
        cf_params = list(set(cf_params))
        c_formula = condition_formula
        c_formula = condition_form_convert(cf_params,params_info, c_formula)
        #print(c_formula)
        for m in cf_params:
            if m != '<>':
                val = str(params_info[params_info['code']==m]['P_value'].to_list()[0])                
                c_formula = c_formula.replace(str(m),val)
                #print(c_formula,str(m),val)
        c_formula = c_formula.replace(repr('nan'),'-100')
        
        if c_formula.lower() != 'false' and formula_df['type_flag'].to_list()[l] != 'R' and 'nan' not in c_formula.lower() and 'none' not in c_formula.lower():
            #print(c_formula)
            try:
                cf_result = pd.read_sql_query(repr('select '+c_formula)[1:-1],con=engine)
                cf_result = cf_result.loc[0,'?column?']
            except:
                cf_result = False
        else:
            cf_result = False
        if cf_result==True:
            #print('Ind',ind,l)
            ind.append(l)
        cff_formula.append(condition_formula)
        cc_formula.append(c_formula)
        ccf_result.append(cf_result)
        cr_rate.append(formula_df.loc[:,'rate_formula'].to_list()[l])
        
    formula_df.reset_index(drop=True, inplace=True)
    summary_df = pd.DataFrame(list(zip(cff_formula,cc_formula, ccf_result,cr_rate)), columns =['Condition Formula', 'Condition Result','Result','Rate Formula']) 
    return [ind,summary_df]

In [14]:
def condition_form_convert(cf_params,params_info, c_formula):
    for m in cf_params:
        #print(m)
        if m != '<>':
            val = str(params_info[params_info['code']==m]['P_value'].to_list()[0])
            # Change '' to \'\' in case of string val
            if val.isdigit()==False:
                if '.' in val:
                    v = val.replace('.','')
                    if '-' in v:
                        v = v.replace('-','')
                    if v.isdigit():
                        val=val
                    else:
                        val=repr(val)
                else:
                    val=repr(val)
            else:
                val=val
            c_formula = c_formula.replace(str(m),val)
    return c_formula
'''
    else:
        c_formula = c_formula.replace('<>','!=')

# Change '=' to '=='
if '<=' in c_formula:
    c_formula=c_formula.replace('<=','@')
if '>=' in c_formula:
    c_formula=c_formula.replace('>=',';')
if '!=' in c_formula:
    c_formula=c_formula.replace('!=','?')
if '=' in c_formula:
    c_formula = c_formula.replace('=','==')
if '@' in c_formula:
    c_formula=c_formula.replace('@','<=')
if ';' in c_formula:
    c_formula=c_formula.replace(';','>=')
if '?' in c_formula:
    c_formula = c_formula.replace('?','!=')
return c_formula
'''

"\n    else:\n        c_formula = c_formula.replace('<>','!=')\n\n# Change '=' to '=='\nif '<=' in c_formula:\n    c_formula=c_formula.replace('<=','@')\nif '>=' in c_formula:\n    c_formula=c_formula.replace('>=',';')\nif '!=' in c_formula:\n    c_formula=c_formula.replace('!=','?')\nif '=' in c_formula:\n    c_formula = c_formula.replace('=','==')\nif '@' in c_formula:\n    c_formula=c_formula.replace('@','<=')\nif ';' in c_formula:\n    c_formula=c_formula.replace(';','>=')\nif '?' in c_formula:\n    c_formula = c_formula.replace('?','!=')\nreturn c_formula\n"

In [15]:
def calculate_rates(true_df,params_info):
    t_params = []
    for q in range(len(true_df)):
        r_formula = true_df['rate_formula'].to_list()[q]
        for k in re.split(r'[()\*\s]\s*',str(r_formula)):
            if '<' in k and '>' in k:
                t_params.append(k[k.find('<'):(k.find('>')+1)])

    t_params = list(set(t_params))
    #print('t_params',t_params)
    rates=[]
    for q in range(len(true_df)):
        r_formula = true_df['rate_formula'].to_list()[q]
        for m in t_params:
            r_formula = r_formula.replace(str(m),str(params_info[params_info['code']==m]['P_value'].to_list()[0]))
        
        #print('r_formula',r_formula)
            
        #rate = eval(r_formula.lower())
        rate = pd.read_sql_query(repr('select '+str(r_formula))[1:-1],con=engine)
        rate = rate.loc[0,'?column?']
        #print(rate)
        rates.append(rate)        
    true_df['Rate']=rates
    return true_df

In [16]:
def tbo_last_rcpt(regn_no,before):
    tbo_df = vt_tax_based_on.loc[vt_tax_based_on['regn_no']==regn_no]
    tbo_df= copy.deepcopy(tbo_df)
    #print('TBO shape',tbo_df.shape)
    #print(tbo_df.shape)
    #tbo_df = tbo_df.reset_index(drop=True, inplace=True)
    tbo_df['op_dt'] = tbo_df.loc[:,'op_dt'].apply(lambda x: datetime.strptime(x[:10], '%Y-%m-%d'))
    tbo_df = tbo_df.sort_values(by='op_dt', ascending=False)
    #tbo_df = tbo_df.reset_index(drop=True, inplace=True)
    if len(tbo_df)>0:#print(tbo_df.shape)
        for n in tbo_df.index:
            x = before-tbo_df.loc[n,'op_dt']
            #x = x.years*365 + x.months*30 + x.days
            if x.days>=0:
                last_rcpt = tbo_df.loc[n,'rcpt_no']
                break
            last_rcpt=''
        if last_rcpt=='':
            print('No prior receipt info!!! Rcpt:',' Regn No:',regn_no)
            return 0
        else:
            return last_rcpt
    else:
        return 0

In [17]:
def owner_or_tbo(regn_no,params_info,param_info_owner,ot_df,tab,new_vch,rcpt_dt,tax_mode):
    for pi in range(len(param_info_owner)):
        col = param_info_owner['column_name'].to_list()[pi][1:-2]
        par = param_info_owner['column - slab_field'].to_list()[pi]
        code = param_info_owner['code'].to_list()[pi]
        
        p_value = ''
        p_date = 'purchase_dt'
        #print(par,code,tab)
        '''
        if tab=='own':
            p_date = 'Purchase Dt'
            col= col.replace('_',' ').title()
        else:
            p_date= 'purchase_dt'
        '''
        purchase_dt = ot_df[p_date].to_list()[0]
        
        p_valid_date = purchase_dt[:2].isdigit()
        if p_valid_date==False and tab=='tbo':
            #print(p_valid_date,tab)
            purchase_dt = vt_owner.loc[vt_owner['regn_no']==regn_no][p_date].to_list()[0]
        p_valid_date = purchase_dt[:2].isdigit()
        #print(purchase_dt)
        if purchase_dt[:4].isdigit() and p_valid_date==True:
            purchase_dt =datetime.strptime(purchase_dt,'%Y-%m-%d')
        elif purchase_dt[3:6].isalpha() and p_valid_date==True:
            purchase_dt =datetime.strptime(purchase_dt,'%d-%b-%Y')
        elif p_valid_date==True:
            purchase_dt =datetime.strptime(purchase_dt,'%d-%m-%Y')
        if par=='vch_age' and p_valid_date==True:
            p_value = relativedelta(rcpt_dt, purchase_dt)
            #print('p_value',p_value)
            p_value = p_value.years
        elif par=='vch_age_months'and p_valid_date==True:
            p_value = relativedelta(rcpt_dt, purchase_dt)
            p_value = (p_value.years *12) + p_value.months
        elif par=='purchase_dt_vch_age'and p_valid_date==True:
            p_value = relativedelta(rcpt_dt, purchase_dt)
            p_value = p_value.years
        elif par=='fin_vch_age'and p_valid_date==True:
            #print(datetime.strptime(ot_df['purchase_dt'].to_list()[0],'%d-%m-%Y'))
            p_value = relativedelta(rcpt_dt, purchase_dt)
            p_value = p_value.years
        elif par=='tax_month_duration':
            p_value = relativedelta(tax_upto, tax_from)
            p_value = p_value.years *12 + p_value.months
        elif par=='payment_date':
            p_value = rcpt_dt
        #elif par=='distance_run_in_month':
        #    p_value = vt_tax_based_on['distance_run_in_quarter']/4
        #elif par=='transaction_pur_cd':
        #    appl_no = vp_appl_rcpt_mapping[vp_appl_rcpt_mapping['rcpt_no']==rcpt_no]['appl_no'].to_list()[0]
        #    if len(va_details[va_details['appl_no']==appl_no][col].to_list())==0:
        #        print(i,'No pur_cd available in va_details')
        #        print('Appl no',appl_no,' !!! Rcpt:',rcpt_no,' Regn No:',regn_no,'vh_class',vh_class)
        #        continue
        #    else:
        #        p_value = va_details[va_details['appl_no']==appl_no][col].to_list()[0]
        #print(p_value)
        elif par=='vch_type':
            p_value = vm_vh_class[vm_vh_class['Vh Class']==vh_class]['Class Type'].to_list()[0]
        elif par=='new_vch':
            p_value = new_vch
        elif par=='tax_mode':
            p_value = tax_mode
        elif par=='tax_due_from_date':
            p_value = tax_from
        elif par=='permit_sub_catg':
            if 'pmt_catg' in ot_df.columns:
                p_value = ot_df['pmt_catg'].to_list()[0]
            else:
                p_value = float("nan")
        elif par=='delay_days':
            #print(rcpt_dt[:10],' ',tax_from)
            p_value = rcpt_dt-tax_from
            #p_value = p_value.years * 365 + p_value.months * 30 + p_value.days
            if p_value.days > 0:
                p_value = p_value.days
            elif p_value == 0:
                p_value = 1
            else:
                p_value = 0
            #print('delayed',p_value)
        elif par == 'delay_months':
            p_value = relativedelta(rcpt_dt, tax_from)
            #print('delayed months',p_value)
            if p_value.days>0:
                bal_m = 1
            p_value = p_value.years * 12 + p_value.months + bal_m
            #print(p_value)
        elif p_valid_date==True:
            #if tab=='own':
            #    col= col.replace('_',' ').title()
            
            p_value= ot_df[col].to_list()[0]
            if col== 'ac_fitted' :
                if p_value == '0':
                    p_value = 'N'
                elif p_value == '1':
                    p_value='Y'
                else:
                    print(False)
        else:
            p_value=0
        #print(col,' ',par,' ',code,' ',p_value,' ',tab)
        params_info.loc[params_info.index[params_info['code'] == code].tolist()[0],'P_value']=p_value
        #print(code,p_value)
    return params_info

In [18]:
def extract_parameters(regn_no,tax_from,rcpt_dt,params_info,param_info_tbo,param_info_owner,param_info_other,new_vch,pur_cd,tax_mode,final_rate):
    if len(param_info_owner)>0:
        owner_df = vt_owner.loc[vt_owner['regn_no']==regn_no]
        owner_df= copy.deepcopy(owner_df)
        #print(owner_df)
    if len(param_info_tbo)>0:
        last_rcpt = tbo_last_rcpt(regn_no,rcpt_dt)
        #print(last_rcpt)
        if last_rcpt !=0:
            tbo_df = vt_tax_based_on.loc[vt_tax_based_on['rcpt_no']==last_rcpt]
            c = tbo_df.columns
            #for j in c :
                #p= j.replace('_',' ').title()
                #tbo_df.rename(columns={j: p},inplace=True)
            tbo_df= copy.deepcopy(tbo_df)
        else:
            tbo_df=[]
    else:
        tbo_df=[]
        #print('else')
    #print(len(tbo_df))
    # Extract parameters from the TBO table
    if len(tbo_df)>0 :
        #print('TBO')
        tab = 'tbo'
        #print('tbo',param_info_tbo)
        params_info = owner_or_tbo(regn_no,params_info,param_info_tbo,tbo_df,tab,new_vch,rcpt_dt,tax_mode)
        #print('TBO',params_info['P_value'].to_list())
    else:
        #print('Owner')
        tab='own'
        #print('own',param_info_owner)
        params_info = owner_or_tbo(regn_no,params_info,param_info_owner,owner_df,tab,new_vch,rcpt_dt,tax_mode)
    #print('Shape',params_info)
    if len(param_info_other)>0:  
        #print('Other')
        #print(param_info_other.shape)
        for pi in range(len(param_info_other)):
            col = param_info_other['column_name'].to_list()[pi][1:-2]
            par = param_info_other['column - slab_field'].to_list()[pi]
            code = param_info_other['code'].to_list()[pi]
            
            #print(null_df)
            #print('Yes!',)
            #print('Other',code,col,par)
            #if math.isnan(params_info[params_info['code']==code]['P_value'].to_list()[0]):
            if pd.isnull(params_info.loc[params_info['code']==code]['P_value']).to_list()[0]==True:
                if col == 'domain_cd':
                    p_value = vt_tax_based_on[vt_tax_based_on['regn_no']==regn_no]['domain_cd'].to_list()
                    if len(p_value)!=0:
                        p_value = p_value[0]
                    #elif len(vt_tax_based_on[vt_tax_based_on['rcpt_no']==tbo_last_rcpt(regn_no,rcpt_dt)]['domain_cd'].to_list())>0:
                    #    p_value = vt_tax_based_on[vt_tax_based_on['rcpt_no']==tbo_last_rcpt(regn_no,rcpt_dt)]['domain_cd'].to_list()[0]
                    else:
                        p_value = None
                        #print('No Domain Cd Available')
                elif col == 'pmt_type':
                    p_value = vt_tax_based_on[vt_tax_based_on['regn_no']==regn_no]['pmt_type'].to_list()
                    if len(p_value)!=0:
                        p_value = p_value[0]
                    #elif len(vt_tax_based_on[vt_tax_based_on['rcpt_no']==tbo_last_rcpt(regn_no,rcpt_dt)]['pmt_type'].to_list())>0:
                    #    p_value = vt_tax_based_on[vt_tax_based_on['rcpt_no']==tbo_last_rcpt(regn_no,rcpt_dt)]['pmt_type'].to_list()[0]
                    else:
                        p_value = None
                        #print('No Permit Type Available')
                elif col == 'owner_cd':
                    p_value = vt_owner[vt_owner['regn_no']==regn_no]['owner_cd'].to_list()[0]
                elif col== 'pur_cd':
                    if pur_cd==57 or pur_cd == 58:
                        p_value=58
                elif col=='tax_amt':
                    p_value = final_rate
                #print('Other',code,col,par)
                params_info.loc[params_info.index[params_info['code'] == code].tolist()[0],'P_value']=p_value  
    return params_info

In [23]:
date_format = "%d-%b-%Y"
df = pd.DataFrame({'a':[]})
def calculate_tax(regn_no,tax_from,tax_upto='2019-10-03', pur_cd = 58,final_rate=0,tax_mode='Y' ,state_cd='AS',off_cd=141,tm='n',result=df):
    rcpt_dt = tax_upto
    #tm='Q'
    if len(vt_owner[vt_owner['regn_no']==regn_no])>0:
        vh_class = vt_owner[vt_owner['regn_no']==regn_no]['vh_class'].to_list()[0]
        result.loc[result['regn_no']==regn_no,'vh_class'] = vh_class
        new_vch = 'N'
        #print(vh_class)
    else:
        #print(i,'NO INFO IN VT_OWNER',regn_no)
        result.loc[result['regn_no']==regn_no,'Remarks'] = 'NO INFO IN VT_OWNER'
        return
        
    #slab_codes = slab_new[(slab_new['vh_class']==vh_class) & (slab_new['pur_cd']==pur_cd) & (slab_new['tax_mode']==tax_mode) & (slab_new['state_cd']==state_cd) & (slab_new['date_from']<tax_from) & (slab_new['date_to']>tax_from) ]['slab_code'].to_list()
    #tax_from = datetime.strptime(tax_from,'%d-%b-%Y')
    #print(vh_class, pur_cd, state_cd,tax_mode, tax_from)
    slab_new2 = slab_new[(slab_new['vh_class']==vh_class) & (slab_new['pur_cd']==pur_cd) &(slab_new['state_cd']==state_cd) & (slab_new['date_to'] > tax_from)]
    tax_modes = list(set(slab_new2['tax_mode'].to_list()))
    #print(slab_new1,tax_modes)
    final_rates = []
    tax_modess = []
    untils = []

    for tax_mode in tax_modes:
        slab_new1 = slab_new2[slab_new2['tax_mode']==tax_mode]
        #print(tax_mode,slab_new1)
        slab_codes = slab_new1['slab_code'].to_list()
        #print(pur_cd,tax_mode,'Slab Codes Applicable:',slab_codes)
        if len(slab_codes)==0:
            #print(i,'Missed',regn_no,'vh_class',vh_class)
            continue

        formula_df = slab_add_new.loc[(slab_add_new['slab_code'].isin(slab_codes)) & (slab_add_new['state_cd']==state_cd)]
        formula_df = formula_df[formula_df['condition_formula']!='0=0']
        #formula_df = formula_df[formula_df['condition_formula']!='1=1']
        #formula_df.columns

        #print('formula_df',formula_df['condition_formula'])

        #Identify the parameters required for calculating the formulas
        params = []
        for j in range(len(formula_df)):
            formula = formula_df['condition_formula'].to_list()[j] +' '+str(formula_df['rate_formula'].to_list()[j])
            for k in re.split(r'[()\*\s]\s*',formula):
                if '<' in k and '>' in k:
                    params.append(k[k.find('<'):(k.find('>')+1)])
        params = list(set(params))
        #print('Required Parameters :',params)
        params_info = slab_fields.loc[slab_fields['code'].isin(params)]
        params_info = params_info.reindex(columns = params_info.columns.tolist() + ['P_value'])
        #print(params_info)
        param_info_owner = params_info[params_info['def_table']=='vt_owner']
        #print('Owner',param_info_owner['column - slab_field'].to_list())
        param_info_other = params_info[params_info['def_table']!='vt_owner']
        #print('Other',param_info_other['column - slab_field'].to_list())
        #param_info_tbopi = params_info[params_info['table - slab_field']=='vt_tax_based_on_permit_info']
        param_info_tbo =  params_info[params_info['table - slab_field']=='vt_tax_based_on']
        #print('TBO',param_info_tbo['column - slab_field'].to_list())
        #print('Initial Shape',params_info.shape)
        params_info = extract_parameters(regn_no,tax_from,rcpt_dt,params_info,param_info_tbo,param_info_owner,param_info_other,new_vch,pur_cd,tax_mode,final_rate)
        #print('Final Shape',params_info.shape)                                                                        
        #if pur_cd ==58:        
        #print(params_info)
        ind = calculate_formula(formula_df,params_info)[0]
        summary_df = calculate_formula(formula_df,params_info)[1]
        #if pur_cd==57:
        #print(summary_df)
        if len(ind)==0:
            #print('No True Condition')
            continue
        #Module for calculation of rates for true conditions
        true_df = formula_df.loc[ind,:]
        true_df.reset_index(drop=True, inplace=True)
        #print('True_df',true_df)

        #print(t_params)
        true_df_t = true_df[(true_df['type_flag']=='T') | (true_df['type_flag']=='M') | (true_df['type_flag']=='X') ]

        true_df_r = true_df[true_df['type_flag']=='R']
        true_df_s = true_df[true_df['type_flag']=='S']

        true_df_t = calculate_rates(true_df_t,params_info)
        #print('T',true_df_t)
        #print('S',true_df_s)
        #print(rates)
        #print('True_df',true_df_t)

        t_df = true_df_t[(true_df_t['type_flag']=='T')]
        t_df.reset_index(drop=True, inplace=True)               
        #print(t_df)
        if len(t_df)>0:
            #t_rate = true_df_t[true_df_t['type_flag']=='T']['Rate'].to_list()[0]
            #print('t_rate',t_rate)
            #if len(t_df)>1:
            t_rate = 0
            upt = datetime.strptime('01-Jan-1900','%d-%b-%Y')
            #until = max(slab_new1[slab_new1['slab_code'] in t_df['slab_code'].to_list() ]['date_to'].to_list())
            for t in range(len(t_df)):
                sc = t_df.loc[t,'slab_code']
                f_dt = slab_new1.loc[slab_new['slab_code']==sc]['date_from']
                t_dt = slab_new1.loc[slab_new['slab_code']==sc]['date_to']
                fro = max(f_dt.item(),tax_from,upt)   
                upt = min(t_dt.item(),tax_upto)
                n_days = (upt - fro).days + 1
                n_quart = math.ceil(n_days/92) +1
                if pur_cd == 58:
                    if tax_mode=='Q':
                        per_day_rate = t_df.loc[t,'Rate'] / 92
                        until = tax_from + timedelta(n_quart*92)
                    elif tax_mode=='Y5':
                        per_day_rate = t_df.loc[t,'Rate'] / (365*5)
                        until = tax_from + timedelta(365*5)
                    elif tax_mode=='Y10':
                        per_day_rate = t_df.loc[t,'Rate'] / (365*10)
                        until = tax_from + timedelta(365*10)
                    elif tax_mode=='Y15':
                        per_day_rate = t_df.loc[t,'Rate'] / (365*15)
                        until = tax_from + timedelta(365*15)
                    else:
                        per_day_rate = t_df.loc[t,'Rate'] / 365
                        until = tax_from + timedelta(365)
                    t_rate = t_rate + per_day_rate *n_days
                else:
                    t_rate = t_df.loc[t,'Rate']
                #print(pur_cd,t_rate,fro,upt,t_df.loc[t,'Rate'],per_day_rate,n_days,tm)
        #Module for dealing Minimum M and Amount T 
        if len(set(true_df_t['condition_formula'].to_list()))>0:
            #true_df_t = true_df_t[true_df_t['condition_formula']!='1=1']
            if len({'T','M'} - set(true_df_t['type_flag'].to_list()))==0:
                if true_df_t[true_df_t['type_flag']=='T']['Rate'].to_list()[0]> true_df_t[true_df_t['type_flag']=='M']['Rate'].to_list()[0]:
                    final_rate = t_rate
                else:
                    final_rate = true_df_t[true_df_t['type_flag']=='M']['Rate'].to_list()
            elif len({'T','X'} - set(true_df_t['type_flag'].to_list()))==0:
                #print('Here')
                final_rate = true_df_t[true_df_t['type_flag']=='X']['Rate'].to_list()
            else:
                final_rate = t_rate
        #else:       
        #    final_rate = true_df['Rate'].to_list()
        #else:
        #    final_rate = 0
        
        #tax_mode== 'Q'
        #Module for Surcharge
        if len(true_df_s) > 0 :
            params_info.loc[params_info['code'] == '<58>','P_value']=final_rate
            true_df_s = calculate_rates(true_df_s,params_info)
            #print(true_df_s)
            surcharge = true_df_s[true_df_s['type_flag']=='S']['Rate'].to_list()
            #print(final_rate,surcharge)
            final_rate = final_rate + surcharge[0]
        #print(final_rate)
        final_rates.append(final_rate)
        #print(pur_cd,tax_mode,round(final_rate),until)
        if pur_cd ==58:
            tax_modess.append(tax_mode)
            untils.append(until)
            if tax_mode == 'Q':
                result.loc[result['regn_no']==regn_no,'Q_rate'] = final_rate
                result.loc[result['regn_no']==regn_no,'Q_upto'] = until
            elif tax_mode == 'Y':
                #print(final_rate)
                result.loc[result['regn_no']==regn_no,'Y_rate'] = final_rate
                result.loc[result['regn_no']==regn_no,'Y_upto'] = until
            elif tax_mode == 'Y5':
                result.loc[result['regn_no']==regn_no,'Y5_rate'] = final_rate
                result.loc[result['regn_no']==regn_no,'Y5_upto'] = until
            elif tax_mode == 'Y10':
                result.loc[result['regn_no']==regn_no,'Y10_rate'] = final_rate
                result.loc[result['regn_no']==regn_no,'Y10_upto'] = until
            elif tax_mode == 'Y15':
                result.loc[result['regn_no']==regn_no,'Y15_rate'] = final_rate
                result.loc[result['regn_no']==regn_no,'Y15_upto'] = until
            #else:
                #print('Tax mode ',tax_mode,' Rate ',final_rate)
        else:
            result.loc[result['regn_no']==regn_no,'fine'] = final_rate
        
    return result
    '''
    if len(true_df_r) > 0:
        rr_formula = true_df_r[true_df_r['condition_formula']==f]['rate_formula'].to_list()[0]
        ra_formula = condition_form_convert(reb_params,params_info,rr_formula)
        reb_rate_result = eval(ra_formula.lower())              
        final_rate = final_rate - reb_rate_result
         
    #Module for Tax Mode Remaining years
    if tax_mode in ['Y15','Y10','Y12','Y20']:
        tax_period = relativedelta(datetime.strptime(tax_upto, date_format), datetime.strptime(tax_from, date_format))
        tax_period = tax_period.years*365 + tax_period.months*30 + tax_period.days
        #print('Tax Upto - Tax From =',tax_period,'days')
        veh_age = relativedelta(datetime.strptime(tax_from, date_format), datetime.strptime(vt_owner[vt_owner['Regn No']==regn_no]['Purchase Dt'].to_list()[0],'%Y-%m-%d'))
        #print(((veh_age.years * 365) + (veh_age.months * 30) + veh_age.days))
        if tax_mode=='Y10':
            days_rem = 3652 - ((veh_age.years * 365) + (veh_age.months * 30) + veh_age.days)
            final_rate = (final_rate/3652)*days_rem
        if tax_mode=='Y12':
            days_rem = 4383 - ((veh_age.years * 365) + (veh_age.months * 30) + veh_age.days)
            #print('Days remaining in ',tax_mode,'=',days_rem,'days')
            final_rate = (final_rate/4383)*days_rem
        if tax_mode=='Y15':
            days_rem = 5478 - ((veh_age.years * 365) + (veh_age.months * 30) + veh_age.days)
            final_rate = (final_rate/5478)*days_rem
    '''                            
    #Module for Quarterly Mode
    '''
    if tax_mode== 'Q':
        #print(tax_upto,' ',tax_from)
        tax_period = relativedelta(tax_upto, tax_from)
        #print(tax_period)
        years_q = tax_period.years * 4 
        #tax_period = tax_period.years*12 + tax_period.months
        if tax_period.months <=3:
            months_q = 1
        elif tax_period.months <=6:
            months_q = 2
        elif tax_period.months <=9:
            months_q = 3
        else:
            months_q = 4
        n_quart = years_q + months_q
        #print('n_quart',n_quart,final_rate,final_rate*n_quart)
        final_rate = final_rate * n_quart
    elif tax_mode == 'Y':
        tax_period = relativedelta(tax_upto, tax_from)
        years_y = tax_period.years * 12 
        months_y = math.ceil(tax_period.months + years_y)
        if tax_period.days > 0:
            months_y = months_y+1
        #print((final_rate / 12),months_y)
        if tm == 'y2q':
            months_to_quart = math.ceil(months_y/3)
            final_rate = (final_rate / 4) * months_to_quart
        else:
            final_rate = (final_rate / 12) * months_y
    '''
    

In [20]:
def_list = pd.read_csv(r'D:\GA\Defaulters List Goa-1.csv')
def_list.drop_duplicates(subset ="Regn No", keep = 'last', inplace = True)
#def_list = pd.read_excel(r'D:\GA\Def List (Hist dups removed).xlsx')
#def_list = def_list[def_list['vh_class']==59]
def_list = def_list[def_list['Regn No']=='GA05T2486']
#def_list=def_list[(def_list['tax_amt']==900) & (def_list['tax_mode']=='Y')]
#def_list = def_list[def_list['Tax Arrear From'].notnull()]
#def_list.columns
def_list.shape

C:\Users\CDMA\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (2,3,4,5,6,7,10,11,12,13,14,18,19,20,73,74) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(1, 85)

In [24]:
nc_regn_no_list = []
result_df = pd.DataFrame({'regn_no': [0], 
                           'vh_class': [0],
                           'latest_tax_upto': [0],
                           'Q_rate':[0],
                           'Q_upto':[0],
                           'Y_rate':[0],
                           'Y_upto':[0],
                           'Y5_rate':[0],
                           'Y5_upto':[0],
                           'Y10_rate':[0],
                           'Y10_upto':[0],
                           'Y15_rate':[0],
                           'Y15_upto':[0],
                           'fine':[0],
                           'Remarks':['Nil'],
                           'Report Amt':[0],
                           'Report Upto':[0],
                           'Report Fine':[0]
                           
                          })
for i in range(0,1):  
    regn_no = def_list['Regn No'].to_list()[i]    
    print('case',i,regn_no)
    state_cd = state
    #print(regn_no,state)
    #off_cd = 1
    vh_class = vt_owner[vt_owner['regn_no']==regn_no]['vh_class'].to_list()[0]
    #rcpt_no = def_list['Rcpt No'].to_list()[i]
    #print(regn_no,rcpt_no)
    #tax_amt = def_list['Tax Amt'].to_list()[i]
    #rcpt_dt = def_list['Rcpt Dt'].to_list()[i]
    
    #tax_from = def_list['latest_tax_paid_upto'] + 1
    tax_from = def_list['Max Upto'].to_list()[i]
    #print(regn_no,tax_from)
    tax_from = datetime.strptime(str(tax_from)[:10], '%d-%m-%Y')
    
    #tax_upto = def_list['Tax Arrear UpTo'].to_list()[i]
    #tax_upto = datetime.strptime(tax_upto,'%d-%b-%Y')
    tax_upto = datetime.strptime('09-May-2020','%d-%b-%Y')
    #print('Delayed',tax_upto-tax_from)
    result = pd.DataFrame({'regn_no': [regn_no], 
                           'vh_class': [0],
                           'latest_tax_upto': [tax_from],
                           'Q_rate':[0],
                           'Q_upto':[0],
                           'Y_rate':[0],
                           'Y_upto':[0],
                           'Y5_rate':[0],
                           'Y5_upto':[0],
                           'Y10_rate':[0],
                           'Y10_upto':[0],
                           'Y15_rate':[0],
                           'Y15_upto':[0],
                           'fine':[0],
                           'Remarks':['Nil'],
                           'Report Amt':[0],
                           'Report Upto':[0],
                           'Report Fine':[0]
                           
                          })
    #print(result)
    result = calculate_tax(regn_no,tax_from,tax_upto,state_cd=state,result=result)
    #c_tax_amt = result[0]
    #tax_modes = result[1]
    #untils = result[3]
    #print(c_tax_amt)
    #if c_tax_amt == 0:
        #print('Unable to calculate tax')
    #    nc_regn_no_list.append(regn_no)
    #tax_amt = pd.to_numeric(def_list['Tax Amount in (Rs)'].str.replace(',',''),errors='coerce').to_list()[i]
    result.loc[result['regn_no']==regn_no,'Report Amt'] = 0
    #tax_fine = pd.to_numeric(def_list['Tax Fine in (Rs)'].str.replace(',',''),errors='coerce').to_list()[i]
    result.loc[result['regn_no']==regn_no,'Report Fine'] = 0
    
    result_df.append(result, ignore_index=True)
    #print(result)
    #c_tax_fine=0
    c_tax_amt = result['Y_rate'][0]
    fine_result = calculate_tax(regn_no,tax_from,tax_upto, pur_cd = 57, final_rate = c_tax_amt,state_cd=state,result=result) 
    #    c_tax_fine = fine_result[0]
    result_df = result_df.append(fine_result,ignore_index=True)
    print(fine_result)
    #if tax_amt!=c_tax_amt: 
    #    print('Mismatch',i,regn_no, vh_class,str(tax_from)[:11],str(tax_upto)[:11],tax_amt,c_tax_amt,tax_modes,untils,c_tax_fine,tax_fine)
    #else:
    #    print(i, regn_no,vh_class,str(tax_from)[:11],str(tax_upto)[:11], tax_amt, c_tax_amt,c_tax_fine,tax_fine)
    #summary_df = c_tax_amt[1]
    #print('Report',regn_no,tax_amt,tax_fine,str(tax_from)[:11],str(tax_upto)[:11])

#print(result_df.shape)

case 0 GA05T2486
     regn_no  vh_class latest_tax_upto  Q_rate  Q_upto       Y_rate  \
0  GA05T2486        59      2019-03-31       0       0  2002.191781   

                Y_upto  Y5_rate  Y5_upto  Y10_rate  Y10_upto  Y15_rate  \
0  2020-03-30 00:00:00        0        0         0         0         0   

   Y15_upto        fine Remarks  Report Amt  Report Upto  Report Fine  
0         0  500.547945     Nil           0            0            0  


In [44]:
result_df.drop([0])
result_df = pd.merge(result_df,def_list,left_on='regn_no',right_on='Regn No')
result_df.to_csv(path+'Defaulters list with due tax amount-2.csv',index=False)

In [31]:
def_list = def_list[def_list['Tax Amount in (Rs)'].notnull()]
#def_list = def_list[def_list['Vehicle Class']=='Motor Cab']
def_list = def_list[def_list['Tax Arrear From'].notnull()]
def_list = def_list[def_list['Tax Amount in (Rs)'].notnull()]
def_list = def_list[def_list['Tax Type']=='MV Tax']
def_list.reset_index(drop=True, inplace=True)

In [214]:
slab_add_new = pd.read_excel(path+'/vm_tax_slab_add_new.xlsx')

In [40]:
def_list.columns

Index(['Max Upto', 'Table Names', 'tcr_no', 'iss_auth', 'iss_dt', 'rcpt_dt',
       'tax_mode', 'payment_mode', 'tax_amt', 'tax_fine', 'flag',
       'collected_by', 'exem_by', 'perm_dt', 'rcpt_no', 'Date', 'From', 'Upto',
       'Rcpt No', 'remark', 'user_cd', 'Regn No', 'state_cd', 'off_cd',
       'regn_no', 'regn_dt', 'purchase_dt', 'owner_sr', 'owner_name', 'f_name',
       'c_add1', 'c_add2', 'c_add3', 'c_district', 'c_pincode', 'c_state',
       'p_add1', 'p_add2', 'p_add3', 'p_district', 'p_pincode', 'p_state',
       'owner_cd', 'regn_type', 'vh_class', 'chasi_no', 'eng_no', 'maker',
       'maker_model', 'body_type', 'no_cyl', 'hp', 'seat_cap', 'stand_cap',
       'sleeper_cap', 'unld_wt', 'ld_wt', 'gcw', 'fuel', 'color', 'manu_mon',
       'manu_yr', 'norms', 'wheelbase', 'cubic_cap', 'floor_area', 'ac_fitted',
       'audio_fitted', 'video_fitted', 'vch_purchase_as', 'vch_catg',
       'dealer_cd', 'sale_amt', 'laser_code', 'garage_add', 'length', 'width',
       'height', 